# Matching Networks(Metrics Based Method): Omniglot Dataset

In this tutorial, we will learn how to create Matching Networks, and train it on Omniglot Dataset.

###### To begin, Let's first understand what Omniglot dataset is:

The Omniglot data set is designed for developing more human-like learning algorithms. It contains 1623 different handwritten characters from 50 different alphabets. Each of the 1623 characters was drawn online via Amazon's Mechanical Turk by 20 different people. Each image is paired with stroke data, a sequences of [x,y,t] coordinates with time (t) in milliseconds. For more details, please refer : https://github.com/brendenlake/omniglot

#### Matching Networks Architecture

Matching networks, in general, proposes a framework which learns a network that maps a small training dataset and test unlabeled example to same embeddings space. Matching networks aim to learn the proper embeddings representation of small training dataset and use differentiable k-NN with cosine similarity measure to ensure whether a test data point is something ever seen or not.

Matching networks are designed to be two-fold:

Modeling level: At Modeling level, they proposed Matching nets, which uses advances made in attention and memory that enable fast and efficient learning.

Training procedure: At Training Level, they have one condition that distribution of training and test set must be the same. For example: show a few examples per class, switching the task from minibatch to minibatch, similar to how it will be tested when presented with a few examples of a new task.


![Screen%20Shot%202019-05-15%20at%2011.10.18%20PM.png](Images/MatchingNetworks.png)


As we have learned, Matching Networks Architecture implementation consists of following 5 important parts:

1. Embeddings Extractor, g.
2. Full Context Embeddings, Bi-directional LSTM, f
3. Cosine Similarity Distance Function, c
4. Attention Model: Softmax(c)
5. Loss Function: Cross Entropy Loss

##### Step 1: Import all necessary libraries 

In [1]:
# !pip install tqdm
# !pip install matplotlib

In [2]:
import numpy as np
import torch
import torch.nn as nn
import math
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
import tqdm
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
%matplotlib inline 

#### Step 2: We will load omniglot dataset, tranformed in .npy format using helper script.  
In helper script, we are just loading data in size format: [total_number,character,28,28].                             
for more details, go through helper.py script.

In [3]:
batch_size = 16 # setting batch_size
classes_per_set = 20 # Number of classes per set
samples_per_class = 1 # as we are choosing it to be one shot learning, so we have 1 sample

In [4]:
x = np.load('../../originalDataset/omniglot-master/python/dataset.npy') # Load Data
print (x.shape)
print (type(x))
n_classes = x.shape[0] # total number of classes
x = np.reshape(x, newshape=(x.shape[0], x.shape[1], 28, 28, 1)) # expand dimension from (x.shape[0],x.shape[1],28,28)
np.random.shuffle(x) # shuffle dataset
x_train, x_val, x_test = x[:1200], x[1200:1411], x[1411:] # divide dataset in to train, val,ctest

print (x_train.shape)

(1623, 20, 28, 28)
<class 'numpy.ndarray'>
(1200, 20, 28, 28, 1)


##### Preprocess Images: Here we have use normalization method. 

In [5]:
def processes_batch(data, mu, sigma):
    return (data - mu) / sigma

# Normalize Dataset
x_train = processes_batch(x_train, np.mean(x_train), np.std(x_train))
x_val = processes_batch(x_val, np.mean(x_val), np.std(x_val))
x_test = processes_batch(x_test, np.mean(x_test), np.std(x_test))

# Defining dictionary of dataset
datatset = {"train": x_train, "val": x_val, "test": x_test}

Let's Visualize example of one character written by 20 people.

In [16]:
# temp = x_train[5,:,:,:,:]  
# for i in range(0,20):
#     plt.figure()
#     plt.imshow(temp[i,:,:,0])

###### Step 3: Training Data Processing
To Load Omniglot dataset, and prepare it for Matching Networks Architecture, we need to create:
1. Label Set: Variable choose_label 
2. Support Set: support_set_x, support_set_y
3. Batch from Suppport Set Examples

Let's first create a batch which can give a support set, and target set.

In [7]:
def sample_batch(data):
    """
    Generates sample batch 
    :param : data - one of(train,test,val) our current dataset shape [total_classes,20,28,28,1]
    :return: [support_set_x,support_set_y,target_x,target_y] for Matching Networks

    每一个batch，都重复做batch_size次这样的操作：
    1. 从一千多个class里面选择20个，是support_set_x。每个类都只有1个图片，每个图片是28*28*1。
    2. 这20个class对应的support_set_y就是0～19.
    3. 从这20个class里面选择出1个作为target_x, target_y。
    嗯差不多就是这样。
    """
    support_set_x = np.zeros((batch_size, classes_per_set, samples_per_class, data.shape[2],
                              data.shape[3], data.shape[4]), np.float32)
    support_set_y = np.zeros((batch_size, classes_per_set, samples_per_class), np.int32)

    target_x = np.zeros((batch_size, data.shape[2], data.shape[3], data.shape[4]), np.float32)
    target_y = np.zeros((batch_size, 1), np.int32)
    for i in range(batch_size):
        choose_classes = np.random.choice(data.shape[0], size=classes_per_set, replace=False) # choosing random classes
        choose_label = np.random.choice(classes_per_set, size=1) # label set
        choose_samples = np.random.choice(data.shape[1], size=samples_per_class + 1, replace=False)
        x_temp = data[choose_classes] # choosing classes
        x_temp = x_temp[:, choose_samples] # choosing sample batch from classes chosen outputs (20X2X28X28X1)
        y_temp = np.arange(classes_per_set) # will return [0,1,2,3,...,19]
        support_set_x[i] = x_temp[:, :-1] ## (20X前面那1个X28*28*1)
        support_set_y[i] = np.expand_dims(y_temp[:], axis=1) # expand dimension ## shape will be (20,1)
        target_x[i] = x_temp[choose_label, -1] ## (1X后面那1个X28X28X1)
        target_y[i] = y_temp[choose_label] ## shape = (1,)
    return support_set_x, support_set_y, target_x, target_y # returns support of [batch_size, 20 classes per set, 1 sample, 28, 28,1]
    
def get_batch(dataset_name):
    """
    gen batch while training
    :param dataset_name: The name of dataset(one of "train","val","test")
    :return: a batch images

    实际上本质上还是sample_batch操作，只不过多了一步，就是把某一个形状是1的维度消去罢了。
    """
    support_set_x, support_set_y, target_x, target_y = sample_batch(datatset[dataset_name])
    support_set_x = support_set_x.reshape((support_set_x.shape[0], support_set_x.shape[1] * support_set_x.shape[2],
                                           support_set_x.shape[3], support_set_x.shape[4], support_set_x.shape[5]))
    support_set_y = support_set_y.reshape(support_set_y.shape[0], support_set_y.shape[1] * support_set_y.shape[2])
    return support_set_x, support_set_y, target_x, target_y
    

If you recall, Matching Networks architecture, you will remember that there are mainly 4 parts of network:
1. Embeddings extractor(g)
2. Full-Context Embeddings(f)
2. Attention Model(a)
3. Distance Function(c)



so, In this section first we will create a classifier.

##### Step 4: Create an Embeddings extractor

In [8]:
def convLayer(in_channels, out_channels, dropout_prob=0.0):
    """
    :param dataset_name: The name of dataset(one of "train","val","test")
    :return: a batch images
    """
    cnn_seq = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1),
        nn.ReLU(True),
        nn.BatchNorm2d(out_channels),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout(dropout_prob)
    )
    return cnn_seq

class Embeddings_extractor(nn.Module):
    def __init__(self, layer_size=64, num_channels=1, dropout_prob=0.5, image_size=28):
        super(Embeddings_extractor, self).__init__()
        """
        Build a CNN to produce embeddings
        :param layer_size:64(default)
        :param num_channels:
        :param keep_prob:
        :param image_size:
        """
        self.layer1 = convLayer(num_channels, layer_size, dropout_prob)
        self.layer2 = convLayer(layer_size, layer_size, dropout_prob)
        self.layer3 = convLayer(layer_size, layer_size, dropout_prob)
        self.layer4 = convLayer(layer_size, layer_size, dropout_prob)

        finalSize = int(math.floor(image_size / (2 * 2 * 2 * 2)))
        self.outSize = finalSize * finalSize * layer_size

    def forward(self, image_input):
        """
        :param: Image
        :return: embeddings
        """
        x = self.layer1(image_input)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size()[0], -1)
        return x


##### step 5: Create an Attention model after classifier.
a(x,x^)= softmax of cosine similarities

In [9]:
class AttentionalClassify(nn.Module):
    def __init__(self):
        super(AttentionalClassify, self).__init__()

    def forward(self, similarities, support_set_y):
        """
        Products pdfs over the support set classes for the target set image.
        :param similarities: A tensor with cosine similarites of size[batch_size,sequence_length]
        :param support_set_y:[batch_size,sequence_length,classes_num]
        :return: Softmax pdf shape[batch_size,classes_num]
        """
        ## 这个的输入的形状是啥呢？
        ## similarities的形状(16,20)，表示的是各个batch里面，目标图片的向量跟support的各个图片的相似程度。 
        ## (16,20,20)  support_set_y则表示了，support图片的label的onehot形式。本质上这个张量，每个batch里面都是一个20维的对角矩阵。
        softmax = nn.Softmax(dim=1)
        softmax_similarities = softmax(similarities)
        preds = softmax_similarities.unsqueeze(1).bmm(support_set_y).squeeze()
        ## unsqueeze之后变成（16，1，20），跟 (16,20,20) 相乘，得到的应该是(16,1,20)，然后再squeeze一下得到（16，20）
        return preds

#### Step 6: Create a Distance Network, which will take output from Test Image, and training embeddings, to calculate the Distance.

find cosine similarities between support set and input_test_image

In [10]:
class DistanceNetwork(nn.Module):
    """
    This model calculates the cosine distance between each of the support set embeddings and 
    the target image embeddings.
    """

    def __init__(self):
        super(DistanceNetwork, self).__init__()

    def forward(self, support_set, input_image):
        """
        forward pass
        :param support_set:the embeddings of the support set images.shape[sequence_length,batch_size,64]
        :param input_image: the embedding of the target image,shape[batch_size,64]
        :return:shape[batch_size,sequence_length]
        """
        ## 输入的形状，我们分别设定为 (20, 16, 64) (16, 64)
        eps = 1e-10
        similarities = []
        for support_image in support_set: ## support_image的形状(16,64)
            sum_support = torch.sum(torch.pow(support_image, 2), 1) ## (16,)
            support_manitude = sum_support.clamp(eps, float("inf")).rsqrt() ## rsqrt，就是（开方然后一个倒数操作）
            dot_product = input_image.unsqueeze(1).bmm(support_image.unsqueeze(2)).squeeze()
            ## input_image unsqueeze之后是（16，1，64），support_image.unsqueeze(2)之后是(16,64,1),
            ## bmm是batch matmul的意思，所以得到(16, 1)，再squeeze一下就是（16，）了
            cosine_similarity = dot_product * support_manitude ## 这一步乘法，就得到了最后的余弦相似度。形状(16,)
            ## 我横竖看了几遍，觉得这里余弦相似度的求法，似乎不对，上面这个cosine_similarity应该再
            ## 除以一个input_image的向量的模长。
            ## 但后来我意识到，如果similarity的结果是用来做比较的，那么大可不必再除以一个常数了，因为这不影响比较的结果。
            ## 不过similarity值本身就很重要，那还是要把这个数除上去的。
            similarities.append(cosine_similarity)
        similarities = torch.stack(similarities) ## (20, 16)
        return similarities.t() ## (16, 20) ## 这个similarity就代表了，input_image跟20个class的样本之间的相似程度。

##### Step 7:  Create a Bi-directional LSTM, which is taking input and output from Test-image, and put them in same embeddings space.  
If we wish to use full-context embeddings, Matching Networks introduced Bi-directional LSTM for it.

In [11]:
class BidirectionalLSTM(nn.Module):
    def __init__(self, layer_size, batch_size, vector_dim):
        super(BidirectionalLSTM, self).__init__()
        """
        Initial a muti-layer Bidirectional LSTM
        :param layer_size: a list of each layer'size
        :param batch_size: 
        :param vector_dim: 
        """
        self.batch_size = batch_size
        self.hidden_size = layer_size[0]
        self.vector_dim = vector_dim
        self.num_layer = len(layer_size)
        self.lstm = nn.LSTM(input_size=self.vector_dim, num_layers=self.num_layer, hidden_size=self.hidden_size,
                            bidirectional=True)
        self.hidden = (Variable(torch.zeros(self.lstm.num_layers * 2, self.batch_size, self.lstm.hidden_size),requires_grad=False),
                Variable(torch.zeros(self.lstm.num_layers * 2, self.batch_size, self.lstm.hidden_size),requires_grad=False))

    def repackage_hidden(self,h):
        """Wraps hidden states in new Variables, 
        to detach them from their history."""
        if type(h) == torch.Tensor:
            return Variable(h.data)
        else:
            return tuple(self.repackage_hidden(v) for v in h)
        
    def forward(self, inputs):
        self.hidden = self.repackage_hidden(self.hidden)
        output, self.hidden = self.lstm(inputs, self.hidden)
        return output
    

##### Step 8: Let's club all small modules we made, and create a matching network. 

In [12]:
class MatchingNetwork(nn.Module):
    def __init__(self, keep_prob, batch_size=32, num_channels=1, learning_rate=1e-3, fce=False, num_classes_per_set=20, \
                 num_samples_per_class=1, image_size=28):
        """
        Matching Network
        :param keep_prob: dropout rate
        :param batch_size:
        :param num_channels:
        :param learning_rate:
        :param fce: Flag indicating whether to use full context embeddings(i.e. apply an LSTM on the CNN embeddings)
        :param num_classes_per_set:
        :param num_samples_per_class:
        :param image_size:
        """
        super(MatchingNetwork, self).__init__()
        self.batch_size = batch_size
        self.keep_prob = keep_prob
        self.num_channels = num_channels
        self.learning_rate = learning_rate
        self.num_classes_per_set = num_classes_per_set
        self.num_samples_per_class = num_samples_per_class
        self.image_size = image_size
        # Let's set all peices of Matching Networks Architecture
        self.g = Embeddings_extractor(layer_size=64, num_channels=num_channels, dropout_prob=keep_prob, image_size=image_size)
        self.f = fce # if we are considering full-context embeddings
        self.c = DistanceNetwork() # cosine distance among embeddings
        self.a = AttentionalClassify() # softmax of cosine distance of embeddings
        if self.f:
            self.lstm = BidirectionalLSTM(layer_size=[32], batch_size=self.batch_size, vector_dim=self.g.outSize)

    def forward(self, support_set_images, support_set_y_one_hot, target_image, target_y):
        """
        Main process of the network
        :param support_set_images: shape[batch_size,sequence_length,num_channels,image_size,image_size]
        :param support_set_y_one_hot: shape[batch_size,sequence_length,num_classes_per_set]
        :param target_image: shape[batch_size,num_channels,image_size,image_size]
        :param target_y:
        :return:
        """
        # produce embeddings for support set images
        encoded_images = []
        for i in np.arange(support_set_images.size(1)):
            gen_encode = self.g(support_set_images[:, i, :, :]) ## self.g的输入和输出的维度分别是([16, 1, 28, 28]) ([16, 64])
            encoded_images.append(gen_encode)
        ## 循环结束后，理论上encoded_images的维度应该是(20, 16, 64)
        
        # produce embeddings for target images
        gen_encode = self.g(target_image)## (16, 64)
        encoded_images.append(gen_encode) ## 维度变成了(21, 16, 64)
        output = torch.stack(encoded_images,dim=0) ## 把encoded_images合并成一个向量，形状(21, 16, 64)
        
        # if we are considering full-context embeddings
        if self.f:
            output = self.lstm(output)
            
        # get similarities between support set embeddings and target
        similarites = self.c(
            support_set=output[:-1], ## shape是(20,16,64)
            input_image=output[-1] ## (16,64)
        ) ## (16, 20)，这个similarities就代表了，input_image跟20个class的样本之间的相似程度。
        
        # produce predictions for target probabilities
        preds = self.a(
            similarites, ## (16, 20)
            support_set_y=support_set_y_one_hot ## torch.Size([16, 20, 20])
        ) ## （16，20）；这个preds相当于是给出一个判断，就是各个batch里面，input_image跟20个class里面的哪个是最像的。
        
        # calculate the accuracy
        values, indices = preds.max(1) ## 意思是，在axis=1上做max，然后values就是最大值，indices就是最大值对应的列号。
        accuracy = torch.mean((indices.squeeze() == target_y).float()) ## 这个squeeze看样子没啥用啊。
        ## 得到的accuracy意思就是一个batch里面，预测最像预测对了的比例。
        crossentropy_loss = F.cross_entropy(preds, target_y.long())## 这个就是计算损失了。

        return accuracy, crossentropy_loss

#### Step 9: Create a Dataset Loader.
For our case, as we are using Omniglot Dataset,it will create a Omnligloat builder which calls Matching Network, and run its epochs for training, testing, and validation purpose.

In [13]:
def run_epoch(total_train_batches, name='train'):
    """
    Run the training epoch
    :param total_train_batches: Number of batches to train on
    :return:
    """
    total_c_loss = 0.0
    total_accuracy = 0.0
    for i in range(int(total_train_batches)):
        x_support_set, y_support_set, x_target, y_target = get_batch(name)
        '''
        (16, 20, 28, 28, 1)
        (16, 20)
        (16, 28, 28, 1)
        (16, 1)
        '''
        x_support_set = Variable(torch.from_numpy(x_support_set)).float()
        y_support_set = Variable(torch.from_numpy(y_support_set), requires_grad=False).long()
        x_target = Variable(torch.from_numpy(x_target)).float()
        y_target = Variable(torch.from_numpy(y_target), requires_grad=False).squeeze().long()
        '''
        (16, 20, 28, 28, 1)
        (16, 20)
        (16, 28, 28, 1)
        (16, )
        '''
        # convert to one hot encoding
        y_support_set = y_support_set.unsqueeze(2) ## (16, 20, 1)
        sequence_length = y_support_set.size()[1] ## 20
        batch_size = y_support_set.size()[0] ## 16
        y_support_set_one_hot = Variable(
            torch.zeros(batch_size, sequence_length, 
            classes_per_set).scatter_(2,y_support_set.data,1), requires_grad=False
        ) ## torch.Size([16, 20, 20]) 这一步是转onehot。

        # reshape channels and change order
        size = x_support_set.size() ## (16, 20, 28, 28, 1)
        x_support_set = x_support_set.permute(0, 1, 4, 2, 3) ## (16, 20, 1, 28, 28)
        x_target = x_target.permute(0, 3, 1, 2) ## (16, 1, 28, 28, )
        acc, c_loss = matchNet(x_support_set, y_support_set_one_hot, x_target, y_target)
        if name == 'train':
            # optimize process
            optimizer.zero_grad()
            c_loss.backward()
            optimizer.step()

        iter_out = "tr_loss: {}, tr_accuracy: {}".format(c_loss, acc)
        total_c_loss += c_loss
        total_accuracy += acc

    total_c_loss = total_c_loss / total_train_batches
    total_accuracy = total_accuracy / total_train_batches
    return total_c_loss, total_accuracy

###### Set-up Experiments variables.

In [14]:
batch_size=20
num_channels=1
lr=1e-3
image_size=28
classes_per_set=20
samples_per_class=1
keep_prob=0.0
fce=True
optim="adam"
wd=0
matchNet = MatchingNetwork(keep_prob, batch_size, num_channels, lr, fce, classes_per_set,
                                samples_per_class, image_size)
total_iter = 0
total_train_iter = 0
optimizer = torch.optim.Adam(matchNet.parameters(), lr=lr, weight_decay=wd)
scheduler = ReduceLROnPlateau(optimizer, 'min',verbose=True)
# Training setup
total_epochs = 20
total_train_batches = 100
total_val_batches = 10
total_test_batches = 10

##### Let's Run Experiments !!!!! 

In [15]:
train_loss,train_accuracy=[],[]
val_loss,val_accuracy=[],[]
test_loss,test_accuracy=[],[]


for e in range(total_epochs):
    ############################### Training Step ##########################################
    total_c_loss, total_accuracy = run_epoch(total_train_batches,'train')
    train_loss.append(total_c_loss)
    train_accuracy.append(total_accuracy)
    
    ################################# Validation Step #######################################
    total_val_c_loss, total_val_accuracy = run_epoch(total_val_batches, 'val')
    val_loss.append(total_val_c_loss)
    val_accuracy.append(total_val_accuracy)
    print("Epoch {}: train_loss:{:.2f} train_accuracy:{:.2f} valid_loss:{:.2f} valid_accuracy:{:.2f}".
          format(e, total_c_loss, total_accuracy, total_val_c_loss, total_val_accuracy))

Epoch 0: train_loss:2.89 train_accuracy:0.40 valid_loss:2.82 valid_accuracy:0.53
Epoch 1: train_loss:2.77 train_accuracy:0.57 valid_loss:2.76 valid_accuracy:0.59
Epoch 2: train_loss:2.71 train_accuracy:0.67 valid_loss:2.68 valid_accuracy:0.68
Epoch 3: train_loss:2.68 train_accuracy:0.72 valid_loss:2.69 valid_accuracy:0.66
Epoch 4: train_loss:2.65 train_accuracy:0.73 valid_loss:2.66 valid_accuracy:0.70


KeyboardInterrupt: 

Now Let's obtain our test accuracy by running the following code block:

In [ ]:
total_test_c_loss, total_test_accuracy = run_epoch(total_test_batches,'test')
print("test_accuracy:{}%".format(total_test_accuracy*100))

#### Step 10: Let's visualize our results 

In [ ]:
def plot_loss(train,val,name1="train_loss",name2="val_loss",title=""):
    plt.title(title)
    plt.plot(train, label=name1)
    plt.plot(val, label=name2)
    plt.legend()

plot_loss(train_loss,val_loss,"train_loss","val_loss","Loss Graph")

In [ ]:
plot_loss(train_accuracy,val_accuracy,"train_accuracy","val_accuracy","Accuracy Graph")